# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

import os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10810'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10810'

print(os.environ.get("OPENAI_API_KEY"));

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [14]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图显示了一个自然景观，其中主要特色是一条穿过绿色草地的木板路。图中的草地呈现出鲜明的绿色，周围布满了各种草本植物和灌木，展现出一片生机勃勃的景象。天空中有蓝色的天空和点缀的白云，增添了一种宁静和平和的氛围。这条木板路似乎引导着观者进入更深入的自然探索之旅，引人思考这条路可能通向何处。整体上，这是一幅展示自然之美与人工建设和谐共存的美丽图画。', role='assistant', function_call=None, tool_calls=None))


In [15]:
response.choices[0].message.content

'这幅图显示了一个自然景观，其中主要特色是一条穿过绿色草地的木板路。图中的草地呈现出鲜明的绿色，周围布满了各种草本植物和灌木，展现出一片生机勃勃的景象。天空中有蓝色的天空和点缀的白云，增添了一种宁静和平和的氛围。这条木板路似乎引导着观者进入更深入的自然探索之旅，引人思考这条路可能通向何处。整体上，这是一幅展示自然之美与人工建设和谐共存的美丽图画。'

### 封装成一个函数 query_image_description

In [16]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [17]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图显示的是两个卡通形象，它们将狗的头部与人类的身体进行了合成。左边的形象是一只拥有巨大肌肉的狗，右边则是普通体型的狗坐着。图中还包含了一些文字，对比了这两种不同状态的壮硕程度和其他特征。

左侧的狗被称为“16岁的我”，伴随着文字“我刚适应一片光明，关系网通，人情世故的大神”，描绘了一个在年轻时候无比强大和无忧无虑的形象。

右边的狗被称为“工作后的我”，其描述为“好累啊，好想摸摸你”，显示了一个工作后感到疲惫和需要慰藉的状态。“我不行你忽悠我，我行也别指望我”，传达了在职场上的压力和对自身能力的认识。

整体来


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [18]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [19]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图展示了从1980年到2020年美国、中国、日本和德国的国内生产总值（GDP）对比。图中，不同的线条代表了各国的GDP走势：

- 蓝色线代表美国，我们可以看到其GDP增长相对平稳且持续增长。
- 红色线代表中国，中国的GDP从1990年代末开始显著增长，并在后期显示出迅猛的增速。
- 紫色线代表日本，日本的GDP在1990年左右达到高峰后，呈现较为平稳或略有下跌的趋势。
- 绿色线代表德国，其GDP增长较为平稳，总体增长幅度较小。

此图有效地展示了这四个主要经济体在过去四十年间的经济增长情况。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [20]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片中显示的是一张笔记本的一页，记录了关于深度学习模型优化技术的内容。主要内容包括：

1. **Prompt Tuning (FMT & Small Model)**: 提到使用了Prompt Tuning技术，适用于小型模型，通过调整input embeddings的方式，即在原始输入序列（\[X_1, X_2, ..., X_n\]）前添加一些token来指导模型学习。

2. **Prefix Tuning**: 是另一种参数调整技术，其中调整的是Transformer模型的一部分权重W，通过分离出可训练的前缀权重 \(W_p\) 与不变的权重 \(W\) 来优化模型。

3. **LoRA (Low-Rank Adaptation)**: 记录了一种名为LoRA的微调技术，该方法通过添加低秩矩阵 \(\Delta W\) 来调整原始权重矩阵 \(W\)，从而达到提升模型性能与减小训练成本的目的。

4. 末尾还提到了关于LoRA的一些实验结果，如使用LoRA模型的内存需求量等，如“LAMA-65GB 使用LoRA-78GB” 和 “QLORA使用的内存为48GB”。

这些笔记说明作者在研究和应用深度学习的参数微调技术，特别是在NLP和transformer模型上的应用。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [21]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片中显示的是一张笔记本的一页，记录了关于深度学习模型优化技术的内容。主要内容包括：

1. **Prompt Tuning (FMT & Small Model)**: 提到使用了Prompt Tuning技术，适用于小型模型，通过调整input embeddings的方式，即在原始输入序列（\[X_1, X_2, ..., X_n\]）前添加一些token来指导模型学习。

2. **Prefix Tuning**: 是另一种参数调整技术，其中调整的是Transformer模型的一部分权重W，通过分离出可训练的前缀权重 \(W_p\) 与不变的权重 \(W\) 来优化模型。

3. **LoRA (Low-Rank Adaptation)**: 记录了一种名为LoRA的微调技术，该方法通过添加低秩矩阵 \(\Delta W\) 来调整原始权重矩阵 \(W\)，从而达到提升模型性能与减小训练成本的目的。

4. 末尾还提到了关于LoRA的一些实验结果，如使用LoRA模型的内存需求量等，如“LAMA-65GB 使用LoRA-78GB” 和 “QLORA使用的内存为48GB”。

这些笔记说明作者在研究和应用深度学习的参数微调技术，特别是在NLP和transformer模型上的应用。

![](./images/handwriting_1.jpg)

In [22]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片显示的是一本笔记本上的笔记内容，主要围绕自然语言处理（NLP）及其技术特别是变压器（Transformers）模型的训练和优化策略。笔记包含了以下几个部分：

1. **左侧**
    - **标题“自然语言理解”和“条件文本生成”**：这个部分展示了笔记的主题是关于NLP的两个主要领域，即理解和生成。
    - **PEFT/SOTA, PBFT Methods**：PEFT可能是指针对特定任务的前沿技术指标（State Of The Art），PBFT可能是指基于这些指标的方法。
    - **Prompt Tuning, Adapter, Prefix, P-Tuning V1/V2**等词条涉及到模型优化和调整的多种技术。这些技术主要用来在不大幅度改变整个模型结构的情况下，通过微调来适应特定的任务或改进性能。
    - **LAN、GPT, XLHF, ChatGPT, BLOOM,等**：这些都是大型语言模型，用于多种NLP任务。

2. **右侧**
    - **多模态指令微调：Llama (3B)** 和 **LoRA, GLORa, AdapterA**等技术名词，这部分笔记看起来是关于多种技术的集成和比较。
    - **Refine oh in LLMs** 和其下的技术细节描述了LLM（大型语言模型）如何进行精细化处理的技术。
    - **MAM Adapters** 和 **Scaled parallel adapter** 提到了一种可能的新方法或模型架构调整。

笔记详细地记录了多种最新的NLP模型优化技术和方法，旨在探索和比较它们在不同方面的应用和效果。这些笔记可以帮助研究者保持对前沿技术的最新理解，并应用于实际的机器学习项目和研究之中。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

In [24]:
content = query_base64_image_description(image_path="./images/handwriting_2.jpg",prompt="请给我这张图片上的句子想表达什么中心思想。")
print(content)

这段文字的中心思想是关于知识和智慧的重要性，以及它们对于解决问题和应对挑战的帮助。全文可以理解为强调了通过阅读和学习，我们可以获得无形的力量，有助于提高个人修养，更好地理解和处理世界上的复杂问题。此外，还传达了知识因被学习而生效，被应用而显其价值的观点。


### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [25]:

content = query_base64_image_description(image_path="./images/handwriting_2.jpg",prompt="请给我这张图片上的句子想表达什么中心思想。")
display(Markdown(content))


这句话的中心思想是在表达珍惜时光、努力生活的重要性。文字翻译为“珍惜当下这个快乐满足的时光，不要让岁月荒废，让自己被时间推着走。”这表达了一个对时间的积极态度，鼓励人们要珍惜眼前的幸福时光，不要被消极的思想和懈怠的行为所影响，要主动地利用和珍视时间。